In [1]:
# General Imports
import os
import pandas as pd
import numpy as np
from rdkit import Chem
import time
import sys
from pathlib import Path
SELFIES_coder_path = Path("../SELFIES_coder")
sys.path.append(SELFIES_coder_path.as_posix())
import SELFIES_coder as SELFIES_CODER
import selfies as sf

SELFIES_CODER.TEST()

# Inital time
start = time.time()


# Know where user actually is
directory_path = os.getcwd()
print("My current directory is : " + directory_path)
folder_name = os.path.basename(directory_path)
print("My directory name is : " + folder_name)

#load file with smiles codes to train and test (in this case one column is enough)
smifile = ('../Data/training_data_smiles_caffeine.parquet')

data = pd.read_parquet(smifile)


from sklearn.model_selection import train_test_split
smiles_train, smiles_test = train_test_split(data["SMILES"], test_size=0.1, train_size=0.9, random_state=42)
print(smiles_train.shape)
print(smiles_test.shape)


print(data[:10])


data = SELFIES_CODER.get_encoded_SELFIES(data['SMILES'].to_list())

print(data[2])


# characters that are used in given SMILES dataset along with initial and stopping characters
charset = set("".join(list(data[0]))+"!E")
char_to_int = dict((c,i) for i,c in enumerate(charset))
int_to_char = dict((i,c) for i,c in enumerate(charset))
embed = data[3] + 5 #20
print(str(charset))
print(len(charset), embed)

shap = smiles_train.shape[0]
shap1 = smiles_test.shape[0]


import json
json = json.dumps(data[2])
f = open("SELFIES_to_mol_seq.json","w")
f.write(json)
f.close()

import json 
json = json.dumps(data[1])
f = open("mol_seq_to_SELFIES.json","w")
f.write(json)
f.close()

import json

json = json.dumps(char_to_int)

# open file for writing, "w" 
f = open("mol_seq_to_int.json","w")
f.write(json)
f.close()

import json
json1 = json.dumps(int_to_char)

# open file for writing, "w" 
f = open("int_to_mol_seq.json","w")
f.write(json1)
f.close()


#vectorization of molecular sequence code
def vectorize(smiles, shap):
        one_hot =  np.zeros((shap, embed , len(charset)),dtype=np.int8)
        for i,smile in enumerate(smiles):
            #encode the startchar
            one_hot[i,0,char_to_int["!"]] = 1
            #encode the rest of the chars
            for j,c in enumerate(smile):
                one_hot[i,j+1,char_to_int[c]] = 1
            #Encode endchar
            one_hot[i,len(smile)+1:,char_to_int["E"]] = 1
        #Return two, one for input and the other for output
        return one_hot[:,0:-1,:], one_hot[:,1:,:]
smiles_train_ = data[0][:int(shap)]
smiles_test_ = data[0][int(shap):int(shap+shap1)]
X_train, Y_train = vectorize(smiles_train_, shap)
X_test,Y_test = vectorize(smiles_test_, shap1)
print(smiles_train.iloc[0])


#going back from vectorized form to redable string
string_test = "".join([int_to_char[idx] for idx in np.argmax(X_train[0,:,:], axis=1)])
print(string_test)

decoded = SELFIES_CODER.convert_back_to_SEFLIES(string_test, data[1])
print(decoded)

decoded_two = SELFIES_CODER.convert_back_to_SEFLIES(smiles_train_[0], data[1])
print(decoded_two)

print("Correct encoding-decoding: "+str(decoded == decoded_two))

smiles_train_[0]
#Import Keras objects
#LSTM cells special kinds of neural network units that are designed to keep an internal state for longer iterations
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Concatenate
from keras import regularizers
input_shape = X_train.shape[1:]
output_dim = Y_train.shape[-1]
latent_dim = 128
lstm_dim = 128



#encoder-decoder architecture
unroll = False
encoder_inputs = Input(shape=input_shape)
encoder = LSTM(lstm_dim, return_state=True,
                unroll=unroll)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
states = Concatenate(axis=-1)([state_h, state_c])
neck = Dense(latent_dim, activation="relu")
neck_outputs = neck(states)



decode_h = Dense(lstm_dim, activation="relu")
decode_c = Dense(lstm_dim, activation="relu")
state_h_decoded =  decode_h(neck_outputs)
state_c_decoded =  decode_c(neck_outputs)
encoder_states = [state_h_decoded, state_c_decoded]
decoder_inputs = Input(shape=input_shape)
decoder_lstm = LSTM(lstm_dim,
                    return_sequences=True,
                    unroll=unroll
                   )
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_dim, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
#Define the model, that inputs the training vector for two places, and predicts one character ahead of the input
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(model.summary())


from keras.callbacks import History, ReduceLROnPlateau
h = History()
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=15, min_lr=0.000001, verbose=1, min_delta=1e-5) #epsilon=min_delta



from tensorflow.keras.optimizers import RMSprop, Adam
opt=Adam(learning_rate=0.005) #Default 0.001
model.compile(optimizer=opt, loss='categorical_crossentropy')



start_MODEL = time.time()
model.fit([X_train,X_train],Y_train, epochs=200, batch_size=256, shuffle=True, callbacks=[h, rlr], validation_data=([X_test,X_test],Y_test)) #100
end_MODEL = time.time()
print("The time of execution of above program is :", end_MODEL-start_MODEL)


import pickle
f = open("Neural_network_history.pickle","wb")
pickle.dump(h.history, f)


smiles_to_latent_model = Model(encoder_inputs, neck_outputs)

smiles_to_latent_model.save("mol_seq2lat.h5")


latent_input = Input(shape=(latent_dim,))
#reuse_layers
state_h_decoded_2 =  decode_h(latent_input)
state_c_decoded_2 =  decode_c(latent_input)
latent_to_states_model = Model(latent_input, [state_h_decoded_2, state_c_decoded_2])
latent_to_states_model.save("lat2state.h5")


#Last one is special, we need to change it to stateful, and change the input shape
inf_decoder_inputs = Input(batch_shape=(1, 1, input_shape[1]))
inf_decoder_lstm = LSTM(lstm_dim,
                    return_sequences=True,
                    unroll=unroll,
                    stateful=True
                   )
inf_decoder_outputs = inf_decoder_lstm(inf_decoder_inputs)
inf_decoder_dense = Dense(output_dim, activation='softmax')
inf_decoder_outputs = inf_decoder_dense(inf_decoder_outputs)
sample_model = Model(inf_decoder_inputs, inf_decoder_outputs)


#Transfer Weights
for i in range(1,3):
    sample_model.layers[i].set_weights(model.layers[i+6].get_weights())
sample_model.save("samplemodel.h5")



print(sample_model.summary())


end = time.time()


print("The time of execution of above program is :", end-start)

working
My current directory is : C:\Users\aleks\Documents\GitHub\caffeine_new_structures\Neural_network
My directory name is : Neural_network
(295965,)
(32886,)
                                              SMILES
0  C(\C(=O)OC)(S)=C/C([N+1]([O-1])=O)=CN(CCCCCCCC...
1  O=C1N(C(N(C=2N=C(SC3=NC4=C(C=CC=C4)N3CC=5N=NN(...
2       O=C1N(C=2N=C(N(C=2C(N1C)=O)C)SC(=S)N3CCCC3)C
3  C1(N(C(=O)CC1)CCCCNC=2N(C)C=3C(N(C(=O)N(C=3N=2...
4          CN1C2=C(N=C1NCCCNC(=O)C)N(C)C(N(C)C2=O)=O
5     C1(N(C)C(=O)N(C)C=2N(C(N(CCN(C)C)C)=NC1=2)C)=O
6  N=1C2=C(C(=O)N(C(=O)N2C)C)N(C)C=1N(CCCN(C=C([N...
7  C(CCN1C(CCC1=O)=O)NC=2N(C=3C(=O)N(C(=O)N(C=3N=...
8  O=C1N(C2=C(C(N1C)=O)N(C)C(=N2)NCCNC(=O)CSC(=S)...
9  O=C1N(C)C(=O)C=2N(C(=NC=2N1C)SC=3N(CC4=CN(CCCC...
All works correct, encoding leads to the same data during decoding...
{'[N+1]': '#', '[=Branch2]': '$', '[=O]': '%', '[Branch1]': '&', '[/C]': "'", '[#Branch2]': '(', '[#C]': ')', '[O]': '*', '[N]': '+', '[=N+1]': ',', '[N-1]': '-', '[Ring1]': '.', '[=Rin

1157/1157 [==============================] - 26s 23ms/step - loss: 0.0708 - val_loss: 0.0689
Epoch 33/200
1157/1157 [==============================] - 27s 23ms/step - loss: 0.0664 - val_loss: 0.0687
Epoch 34/200
1157/1157 [==============================] - 27s 23ms/step - loss: 0.0647 - val_loss: 0.0651
Epoch 35/200
1157/1157 [==============================] - 26s 23ms/step - loss: 0.0635 - val_loss: 0.0752
Epoch 36/200
1157/1157 [==============================] - 27s 24ms/step - loss: 0.0618 - val_loss: 0.0628
Epoch 37/200
1157/1157 [==============================] - 27s 23ms/step - loss: 0.0604 - val_loss: 0.0600
Epoch 38/200
1157/1157 [==============================] - 27s 23ms/step - loss: 0.0591 - val_loss: 0.0614
Epoch 39/200
1157/1157 [==============================] - 27s 24ms/step - loss: 0.0564 - val_loss: 0.0597
Epoch 40/200
1157/1157 [==============================] - 27s 24ms/step - loss: 0.0608 - val_loss: 0.0582
Epoch 41/200
1157/1157 [==============================] - 2

1157/1157 [==============================] - 28s 24ms/step - loss: 0.0229 - val_loss: 0.0314
Epoch 110/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0249 - val_loss: 0.0409
Epoch 111/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0227 - val_loss: 0.0327
Epoch 112/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0238 - val_loss: 0.0298
Epoch 113/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0279 - val_loss: 0.0290
Epoch 114/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0285 - val_loss: 0.0248
Epoch 115/200
1157/1157 [==============================] - 27s 24ms/step - loss: 0.0231 - val_loss: 0.0239
Epoch 116/200
1157/1157 [==============================] - 26s 22ms/step - loss: 0.0221 - val_loss: 0.0251

Epoch 00116: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 117/200
1157/1157 [==============================] - 27s 24ms/step - lo

Epoch 185/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0093 - val_loss: 0.0097
Epoch 186/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0081 - val_loss: 0.0096
Epoch 187/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0091 - val_loss: 0.0117
Epoch 188/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0089 - val_loss: 0.0384
Epoch 189/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0087 - val_loss: 0.0136
Epoch 190/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0126 - val_loss: 0.0089

Epoch 00190: ReduceLROnPlateau reducing learning rate to 0.0012499999720603228.
Epoch 191/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0046 - val_loss: 0.0068
Epoch 192/200
1157/1157 [==============================] - 28s 24ms/step - loss: 0.0046 - val_loss: 0.0062
Epoch 193/200
1157/1157 [==============================] - 28s 

In [2]:
import pickle #if the history file is empty
f = open("Neural_network_history.pickle","wb")
pickle.dump(h.history, f)